In [40]:
# T 和 C 的比例不均衡，没有ST和LC
import numpy as np
import matplotlib.pyplot as plt
from utils import setup_seed

setup_seed(0)
# 500个x样本
n = 10000
d=  10
x = np.random.randn(n, d)
persudable = 0.1*(np.cos(x[:,:5]).sum(axis=1,keepdims=True)+2.0)
# surething = 0.5*(x.sum(axis=1,keepdims=True))**2
# surething = 0.5*np.cos(x[:,:].sum(axis=1,keepdims=True)+1)
# sleepingdog = 0.5*np.sin(2*x[:,:]).sum(axis=1,keepdims=True)
# lostcause = 0.5*x.sum(axis=1,keepdims=True)**3
lostcause = 0.5*np.sin(x[:,:].sum(axis=1,keepdims=True)+1)
cate = persudable - lostcause

# concat 上述4个变量并且softmax
prob_principle = np.concatenate((persudable, surething, sleepingdog, lostcause), axis=1)
prob_principle = np.exp(prob_principle) / np.sum(np.exp(prob_principle), axis=1, keepdims=True)
print(f'prob_principle.shape:{prob_principle.shape}')
n_principles = np.zeros_like(prob_principle)
sum_principles = np.zeros((1,4))
# 对于每个y，生成50个样本，每个样本对应于y[:,0], y[:,1], y[:,2], y[:,3]中的概率值
# obs_x = x.repeat(50, axis=2)
# print(obs_x.shape)
obs_data = np.zeros((n, 16))
obs_data[:,10] = np.random.binomial(n=1, p=0.1, size=n)
cnt = 0
for i in range(n):
    # t
    obs_data[i,:10] = x[i,:]
    # 属于每层的概率
    obs_data[i,12:16] = prob_principle[i,:]
    if np.argmax(prob_principle[i,:]) == 0:
        obs_data[i,11] = obs_data[i,10]
    elif np.argmax(prob_principle[i,:]) == 1:
        obs_data[i,11] = 1
    elif np.argmax(prob_principle[i,:]) == 2:
        obs_data[i,11] = 1-obs_data[i,10]
    elif np.argmax(prob_principle[i,:]) == 3:
        obs_data[i,11] = 0

prob_principle.shape:(10000, 4)


In [41]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(obs_data, test_size=0.2, random_state=0)
valid_data, test_data = train_test_split(test_data, test_size=0.5, random_state=0)
np.savez('D:\code\ecrank\principle-uplift\data\syntheticv7.npz', train=np.array(train_data), valid=np.array(valid_data), test=np.array(test_data))


In [42]:
num_persuadable = 0
num_surething = 0
num_sleepingdog = 0
num_lostcause = 0

for i in range(obs_data.shape[0]):
    tmp = np.argmax(obs_data[i,12:16])
    if tmp == 0:
        num_persuadable += 1
    elif tmp == 1:
        num_surething += 1
    elif tmp == 2:
        num_sleepingdog += 1
    elif tmp == 3:
        num_lostcause += 1

print(f'treated group num: {obs_data[:,10].sum()}/{obs_data.shape[0]}')
print(f'num_persuadable:{num_persuadable}, num_surething:{num_surething}, num_sleepingdog:{num_sleepingdog}, num_lostcause:{num_lostcause}')

treated group num: 1002.0/10000
num_persuadable:5727, num_surething:498, num_sleepingdog:3281, num_lostcause:494


In [4]:
test_data[18]

array([-1.15596149, -1.54013138, -0.52239227,  0.75534745,  0.520944  ,
        2.2820031 ,  0.6241646 ,  0.18202274, -0.82494562, -1.03549233,
        1.        ,  0.        ,  0.26505032,  0.22633257,  0.27581825,
        0.23279887])

In [20]:
for i in range(n):
    print(prob_principle[i])

[0.236882   0.69280878 0.030216   0.04009322]
[0.42606741 0.16445257 0.24808745 0.16139257]
[2.89592999e-04 9.97453039e-01 2.25736767e-03 3.02591321e-16]
[3.10892243e-16 2.70118808e-12 1.01930203e-16 1.00000000e+00]
[1.04317698e-03 9.94104982e-01 4.85184075e-03 4.23005260e-12]
[0.40518467 0.16209748 0.27539053 0.15732732]
[0.16598496 0.15851165 0.51723852 0.15826487]
[2.01491128e-05 1.93915989e-03 1.16060947e-04 9.97924630e-01]
[0.06182268 0.18241547 0.42881758 0.32694427]
[1.37787745e-02 9.74457109e-01 1.17594894e-02 4.62757434e-06]
[1.50137858e-05 1.56557148e-03 8.34054010e-05 9.98336009e-01]
[0.12541474 0.16482385 0.52622029 0.18354113]
[2.12630095e-10 2.43549984e-07 2.72354339e-10 9.99999756e-01]
[0.33063481 0.54405017 0.0363278  0.08898722]
[3.14919209e-02 9.53627279e-01 1.47705964e-02 1.10203547e-04]
[6.01650296e-04 9.95770954e-01 3.62739602e-03 8.75015590e-14]
[8.74753131e-38 1.00000000e+00 1.29296807e-38 0.00000000e+00]
[9.58821351e-05 9.99011416e-01 8.92702022e-04 2.06989075e-

In [21]:
print(prob_principle[906])

[0.28717392 0.15547953 0.41285713 0.14448941]


In [22]:
np.argmax(prob_principle[906,:])

2

In [23]:
obs_data[906,10]

0.0